### An example implementation of the CAREforMe pipeline using the WESAD dataset

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../care_for_me"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import numpy as np
import pandas as pd
import wesad

from care_for_me import signals


subject_indices = list(range(2, 12)) + list(range(13, 18))
SUBJECTS = [str(i) for i in subject_indices]

In [ ]:
# Format data to be compatible with CAREforMe pipeline
# Only needs to be run once locally 
# Status: COMPLETE, do not re-run
import glob
import os


wesad.reformat_and_save_data()

# Rename Medi_1 and Medi_2 files to Medi1 and Medi2
wesad_path = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe\\data\\WESAD"
source_folder = os.path.join(wesad_path, "formatted")

files_to_rename = glob.glob(os.path.join(source_folder, "*/*Medi_*.csv"))
for file in files_to_rename:
    file_name = file.split("_")
    new_name = f"{file_name[0]}_{file_name[1]}{file_name[2]}_{file_name[3]}"
    os.rename(file, new_name)

In [78]:
print(wesad.generate_labels(threshold="dynamic"))

    subject  Base  TSST  Medi1  Fun  Medi2
0         2     0     1      0    0      0
1         3     0     1      0    0      0
2         4     0     1      1    0      1
3         5     0     1      0    1      0
4         6     0     1      0    1      0
5         7     0     1      0    0      1
6         8     0     1      1    0      0
7         9     0     1      0    1      0
8        10     0     1      0    0      0
9        11     0     1      0    0      0
10       13     1     1      0    0      1
11       14     0     1      0    0      0
12       15     1     1      0    0      0
13       16     1     1      0    0      0
14       17     0     1      0    0      1


In [86]:
# Build pipeline with default preprocessing and feature extraction methods
# Asynchronous directed acyclic graph

# from care_for_me import feature_extractor
# from care_for_me import signal_acquisition
# from care_for_me import signal_preprocessor
# from care_for_me import pipeline
import random

from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Node, Pipeline

from sklearn.svm import SVC

# TODO: WESAD data includes phases labeled Medi_1 and Medi_2. Currently, the filename parsing 
# treats these two phases as the same; need to fix.

# Change wesad_path according to your local file structure
wesad_path = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe\\data\\WESAD"
source_folder = os.path.join(wesad_path, "formatted")
signal_types = [
    signals.Signals.ECG,
    signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

signal_acq = SignalAcquisition(source_folder=source_folder, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()

# TODO: Generate correct labels based on features
labels = [random.choice([0, 1]) for _ in range(len(wesad.SUBJECTS))]
model = SVC()
feature_selector = FeatureSelector(model, feature_names, labels, num_features=3)
estimator_train = Estimator(labels, 0, model, name="SVC training")
estimator_test = Estimator(labels, 1, model, name="SVC testing")
estimator_train_val_test = Estimator(labels, 2, model, name="SVC train-val-test")

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    [signal_acq, signal_preprocessor, feature_extractor]
    # [signal_acq, signal_preprocessor, feature_extractor, feature_selector, estimator_train, estimator_test]
    # [signal_acq, signal_preprocessor, feature_extractor, feature_selector, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()
# fitted_model = out[0]
# y_true = out[1]
# y_pred = out[2]

Running node Signal Acquisition...
Elapsed time for Signal Acquisition: 0.0
Running node Signal Preprocessor...
Elapsed time for Signal Preprocessor: 0.8026401996612549
Running node Feature Extractor...


100%|██████████| 15/15 [00:04<00:00,  3.49it/s]

Elapsed time for Feature Extractor: 4.3027589321136475
